In [1]:
import scipy
scipy.__version__

'1.11.1'

In [2]:
import pickle
with open("/cluster/work/cotterell/shared_datasets/pile_deduped/pythia-deduped-bigrams", "rb") as f:
    x = pickle.load(f)


In [3]:
x

<50277x50277 sparse array of type '<class 'numpy.uint32'>'
	with 486734660 stored elements in Compressed Sparse Row format>

In [12]:
# %%timeit
x[1320, 44205]

53

In [6]:
from tqdm.auto import trange
import numpy as np


CHUNK_SIZE = 50_000_000
VOCAB_SIZE = 50_277

counts = np.memmap(
    "/cluster/work/cotterell/shared_datasets/pile_deduped/pythia-unigrams.bin",
    dtype=np.int64,
    mode="w+",
    shape=(VOCAB_SIZE,),
)
counts[:] = 0

view = np.memmap(
    "/cluster/work/cotterell/shared_datasets/pile_deduped/pythia_pile_idxmaps/pile_0.87_deduped_text_document.bin",
    dtype=np.uint16, 
    mode="r"
)

for i in trange(0, len(view), CHUNK_SIZE):
    chunk = view[i : i + CHUNK_SIZE]
    counts += np.bincount(chunk, minlength=len(counts))

  0%|          | 0/4144 [00:00<?, ?it/s]

In [43]:
np.save(counts, "/cluster/work/cotterell/shared_datasets/pile_deduped/my-pythia-deduped-unigrams.npy")

TypeError: expected str, bytes or os.PathLike object, not memmap

In [16]:
sum(counts)

143820414058

In [11]:
counts_pre = np.load("/cluster/work/cotterell/shared_datasets/pile_deduped/pythia-deduped-unigrams.npy")

In [14]:
sum(counts_pre)

300039168000

In [5]:
CHUNK_SIZE

50000000

In [17]:
counts_nondeduped = np.load("/cluster/work/cotterell/shared_datasets/pile_deduped/pythia-unigrams.npy")

In [25]:
from transformers import AutoTokenizer
MODEL_ID = "EleutherAI/pythia-6.9b-deduped"
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    padding_side="left",
)

In [41]:
tokenizer("Berlin")

{'input_ids': [23666, 3642], 'attention_mask': [1, 1]}